In [ ]:
#Keep relevant variables
df = df[['TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']]

In [143]:
from nba_api.stats.endpoints import leaguegamelog
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [218]:
#Can use this to get means and standard deviations
def generate_game_stats(game_df, teams): 
    #all_stats_arr = np.empty((30, 77, 19))
    all_teams_stats_df = pd.DataFrame()
    for i, team in enumerate(teams):
        temp_df = df[df['TEAM_NAME'] == team]
        temp_df = temp_df.sort_values(by = ['GAME_DATE'])
        temp_df = temp_df.set_index('GAME_ID')
        key_data = temp_df[['GAME_DATE', 'TEAM_NAME', 'MATCHUP', 'WL']].iloc[5:]
        temp_df = temp_df[[
           'FGM', 'FGA', 'FG_PCT', 'FG3M',
           'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
           'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']]
        #final_stats_df = temp_df.rolling(5).mean().shift(periods = 1).iloc[5:]#.to_dict('list')
        #single_stats_arr = final_stats_df.to_numpy()
        #all_stats_arr[i] = single_stats_arr
        
        final_organized_stats = temp_df.rolling(5).mean().shift(periods = 1).iloc[5:]#.to_dict('index')
        stats_with_key = pd.concat([final_organized_stats, key_data], axis = 1)
     
        all_teams_stats_df = pd.concat([all_teams_stats_df, stats_with_key], axis = 0)
        
    home_df = all_teams_stats_df[all_teams_stats_df['MATCHUP'].str.contains("vs.")]
    away_df = all_teams_stats_df[all_teams_stats_df['MATCHUP'].str.contains("@")]
    merged_game_stats_df = home_df.merge(away_df, on = "GAME_ID",suffixes = ("_H", "_A"))
    merged_game_stats_df = merged_game_stats_df.sort_values(by = ['GAME_DATE_H'])
      
    return merged_game_stats_df

In [219]:
#Function to get population mean and standard deviations - might be stupid but I think it makes sense
#Make sure to use this on train and test set separately 
#After this, make these stats for each game their own matrices, so I could do (game_matrix - mean_matrix / std_matrix)
def generate_population_statistics(stats_df):
    game_means = np.empty((stats_df.shape[0], 19))
    game_stds = np.empty((stats_df.shape[0], 19))
    
    for i in range(stats_df.shape[0]):
        inter_stats_df = stats_df[stats_df['GAME_DATE_H'].str.contains(stats_df.iloc[i]['GAME_DATE_H'][0:4])]
        inter_index = stats_df.index[i]
        it1 = np.where(inter_stats_df.index == inter_index)[0][0]
        team_dict = {}
        pop_stats_arr = np.empty((30, 19))
        it2 = it1+1
        team_it = 0
    
        while len(team_dict.keys()) < 30:

            if it1 >= 0:

                if inter_stats_df.iloc[it1]['TEAM_NAME_H'] not in team_dict.keys():  
                    team_dict[inter_stats_df.iloc[it1]['TEAM_NAME_H']] = ''

                    pop_stats_arr[team_it] = inter_stats_df.iloc[it1][['FGM_H', 'FGA_H', 'FG_PCT_H', 'FG3M_H', 'FG3A_H', 'FG3_PCT_H', 'FTM_H',
                   'FTA_H', 'FT_PCT_H', 'OREB_H', 'DREB_H', 'REB_H', 'AST_H', 'STL_H',
                   'BLK_H', 'TOV_H', 'PF_H', 'PTS_H', 'PLUS_MINUS_H']].to_numpy()
                    team_it += 1

                if inter_stats_df.iloc[it1]['TEAM_NAME_A'] not in team_dict.keys():
                    team_dict[inter_stats_df.iloc[it1]['TEAM_NAME_A']] = ''

                    pop_stats_arr[team_it] = inter_stats_df.iloc[it1][['FGM_A', 'FGA_A', 'FG_PCT_A',
                   'FG3M_A', 'FG3A_A', 'FG3_PCT_A', 'FTM_A', 'FTA_A', 'FT_PCT_A', 'OREB_A',
                   'DREB_A', 'REB_A', 'AST_A', 'STL_A', 'BLK_A', 'TOV_A', 'PF_A', 'PTS_A',
                   'PLUS_MINUS_A']].to_numpy()
                    team_it += 1

                it1 -= 1    

            if it2 < inter_stats_df.shape[0]:
                if inter_stats_df.iloc[it2]['TEAM_NAME_H'] not in team_dict.keys():  
                    team_dict[inter_stats_df.iloc[it2]['TEAM_NAME_H']] = ''

                    pop_stats_arr[team_it] = inter_stats_df.iloc[it2][['FGM_H', 'FGA_H', 'FG_PCT_H', 'FG3M_H', 'FG3A_H', 'FG3_PCT_H', 'FTM_H',
                   'FTA_H', 'FT_PCT_H', 'OREB_H', 'DREB_H', 'REB_H', 'AST_H', 'STL_H',
                   'BLK_H', 'TOV_H', 'PF_H', 'PTS_H', 'PLUS_MINUS_H']].to_numpy()
                    team_it += 1

                if inter_stats_df.iloc[it2]['TEAM_NAME_A'] not in team_dict.keys():
                    team_dict[inter_stats_df.iloc[it2]['TEAM_NAME_A']] = ''

                    pop_stats_arr[team_it] = stats_df.iloc[it2][['FGM_A', 'FGA_A', 'FG_PCT_A',
                   'FG3M_A', 'FG3A_A', 'FG3_PCT_A', 'FTM_A', 'FTA_A', 'FT_PCT_A', 'OREB_A',
                   'DREB_A', 'REB_A', 'AST_A', 'STL_A', 'BLK_A', 'TOV_A', 'PF_A', 'PTS_A',
                   'PLUS_MINUS_A']].to_numpy()
                    team_it += 1

                it2 += 1
                
        mean_stats = np.mean(pop_stats_arr, axis = 0)        
        std_stats = np.std(pop_stats_arr, axis = 0)
        
        game_means[i] = mean_stats
        game_stds[i] = std_stats
        
    return game_means, game_stds    
    

In [220]:
#Function to get z scores and finalize attributes as (Home minus Away)
def normalize_and_standardize(stats_df, game_means, game_stds):
   
    home_stats = np.empty((stats_df.shape[0], 19))
    away_stats = np.empty((stats_df.shape[0], 19))
    
    home_stats = stats_df.iloc[:, 0:19]
    away_stats = stats_df.iloc[:, 23:42]
    win_loss_home = stats_df.iloc[:, 22]
    matchup = stats_df.iloc[:, [43, 20]]
    
    #standardize home and away stats 
    home_z = np.divide((home_stats - game_means), game_stds)
    
    away_z = np.divide((away_stats - game_means), game_stds)
    
    final = np.subtract(home_z, away_z)
    
    final = pd.DataFrame(final)

    final = pd.concat([final, win_loss_home], axis = 1)
    
    return final, matchup

In [227]:
#Include num_years previous years plus current season games
def generate_full_train_test(num_years):
    start_year = 2022 - num_years
    all_years_stats_df = pd.DataFrame()
    #For each year, 
    for i in range(start_year, 2022):
        games = leaguegamelog.LeagueGameLog(season = str(i))
        df = pd.DataFrame(games.get_data_frames()[0])
        teams = df['TEAM_NAME'].unique()
        stats_df = generate_game_stats(df, teams)
        
        all_years_stats_df = pd.concat([all_years_stats_df, stats_df], axis = 0)
        
    
    #Split data into train and test and transform
    train_stats = all_years_stats_df.iloc[:int(all_years_stats_df.shape[0]*0.7), :]
    test_stats = all_years_stats_df.iloc[int(all_years_stats_df.shape[0]*0.7):, :]

    game_means_train, game_stds_train = generate_population_statistics(train_stats)
    game_means_test, game_stds_test = generate_population_statistics(test_stats)
    
    #Finalize and prepare train / test
    final_train, matchup_train = normalize_and_standardize(train_stats, game_means_train, game_stds_train)
    final_test, matchup_test = normalize_and_standardize(test_stats, game_means_test, game_stds_test)
    
    #Prepare actual train and test
    train_X = final_train.iloc[:,:19]
    train_Y = final_train.iloc[:,19]

    test_X = final_test.iloc[:, :19]
    test_Y = final_test.iloc[:,19]
    
    return train_X, train_Y, test_X, test_Y

In [228]:
train_X, train_Y, test_X, test_Y = generate_full_train_test(5)

C:\Users\batar\AppData\Local\Temp\ipykernel_8300\1595565284.py:17: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  final = np.subtract(home_z, away_z)
C:\Users\batar\AppData\Local\Temp\ipykernel_8300\1595565284.py:17: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy

In [226]:
train_X

,FGM_H,FGA_H,FG_PCT_H,FG3M_H,FG3A_H,FG3_PCT_H,FTM_H,FTA_H,FT_PCT_H,OREB_H,DREB_H,REB_H,AST_H,STL_H,BLK_H,TOV_H,PF_H,PTS_H,PLUS_MINUS_H
GAME_ID,,,,,,,,,,,,,,,,,,,
0021800068,-1.858243,1.640968,-3.462870,0.201080,0.573655,-0.711188,-0.804507,-0.219714,-1.641395,0.637193,-1.279685,-0.731177,-3.640845,0.281690,-2.056876,-2.693408,-1.546329,-2.066312,-1.812867
0021800077,-1.238829,0.717924,-2.150148,-2.111338,-0.614630,-3.437410,2.042210,1.483069,2.177868,1.699182,-1.010277,0.056244,-1.040241,-0.845070,-2.605376,-0.092876,0.483228,-0.762946,-2.825940
0021800078,-0.344119,0.205121,-0.603550,-0.703779,-0.122926,-0.985966,0.185655,0.823927,-1.232823,-1.168187,2.357314,1.349866,0.965939,0.140845,-0.959875,0.557257,1.256392,-0.445052,-1.412970
0021800081,1.598485,0.456879,1.496713,2.906125,2.428623,1.086481,0.740760,0.490087,0.496176,-0.203105,2.965993,2.313567,0.751024,-0.419099,1.234938,1.234740,2.691862,2.745512,2.983514
0021800086,-0.461861,-1.800883,0.667343,-2.004952,-2.632952,0.284111,0.774308,0.156305,1.296404,0.507615,1.522337,1.526980,-0.517555,-1.687632,-1.570332,0.675143,-1.683067,-0.626512,1.317904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0021800880,-3.576110,-2.755891,-1.713582,2.142944,2.146541,0.345866,0.460759,0.724574,-0.896386,-0.662230,-0.604084,-0.845284,-2.729308,-0.134866,-1.093083,0.365419,-1.633184,-1.916357,-0.325883
0021800874,1.500286,-1.086630,2.139685,0.312085,-0.752416,1.237122,-2.322443,-2.734615,0.086563,-3.035261,-0.076908,-1.759573,0.544957,1.718195,-0.312309,-0.455837,0.518749,-0.109144,2.186725
0021800892,0.623456,-1.194562,1.400681,-1.969763,-2.014168,-0.017299,0.922286,1.578484,-1.795109,0.108629,2.026134,1.673533,3.004606,-2.477273,2.358770,0.000000,-0.522741,0.292125,-0.506346


In [2]:
#Pull and store data
#Modify this to concat yearly statistics for larger training set
games = leaguegamelog.LeagueGameLog(season = '2018')
df = pd.DataFrame(games.get_data_frames()[0])
teams = df['TEAM_NAME'].unique()

In [207]:
#Get rolling averages and join like games into (home, away)
stats_df = generate_game_stats(df, teams)

In [191]:
np.where(stats_df.index == '0021801228')[0][0]

1147

In [177]:
stats_df['GAME_DATE_H'].str.contains(stats_df.iloc[2]['GAME_DATE_H'][0:4]) 

GAME_ID
0021800068     True
0021800078     True
0021800077     True
0021800081     True
0021800084     True
              ...  
0021801225    False
0021801228    False
0021801224    False
0021801229    False
0021801222    False
Name: GAME_DATE_H, Length: 1151, dtype: bool

'0021800078'

In [196]:
#Split data into train and test and transform
train_stats = stats_df.iloc[:int(stats_df.shape[0]*0.7), :]
test_stats = stats_df.iloc[int(stats_df.shape[0]*0.7):, :]

game_means_train, game_stds_train = generate_population_statistics(train_stats)
game_means_test, game_stds_test = generate_population_statistics(test_stats)

In [122]:
#Finalize and prepare train / test
final_train, matchup_train = normalize_and_standardize(train_stats, game_means_train, game_stds_train)
final_test, matchup_test = normalize_and_standardize(test_stats, game_means_test, game_stds_test)

C:\Users\batar\AppData\Local\Temp\ipykernel_8300\1595565284.py:17: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  final = np.subtract(home_z, away_z)
C:\Users\batar\AppData\Local\Temp\ipykernel_8300\1595565284.py:17: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy

In [137]:
#Prepare actual train and test
train_X = final_train.iloc[:,:19]
train_Y = final_train.iloc[:,19]

test_X = final_test.iloc[:, :19]
test_Y = final_test.iloc[:,19]

In [150]:
train_X

,FGM_H,FGA_H,FG_PCT_H,FG3M_H,FG3A_H,FG3_PCT_H,FTM_H,FTA_H,FT_PCT_H,OREB_H,DREB_H,REB_H,AST_H,STL_H,BLK_H,TOV_H,PF_H,PTS_H,PLUS_MINUS_H
GAME_ID,,,,,,,,,,,,,,,,,,,
0021800068,-1.858243,1.640968,-3.462870,0.201080,0.573655,-0.711188,-0.804507,-0.219714,-1.641395,0.637193,-1.279685,-0.731177,-3.640845,0.281690,-2.056876,-2.693408,-1.546329,-2.066312,-1.812867
0021800078,-0.344119,0.205121,-0.603550,-0.703779,-0.122926,-0.985966,0.185655,0.823927,-1.232823,-1.168187,2.357314,1.349866,0.965939,0.140845,-0.959875,0.557257,1.256392,-0.445052,-1.412970
0021800077,-1.238829,0.717924,-2.150148,-2.111338,-0.614630,-3.437410,2.042210,1.483069,2.177868,1.699182,-1.010277,0.056244,-1.040241,-0.845070,-2.605376,-0.092876,0.483228,-0.762946,-2.825940
0021800081,1.598485,0.456879,1.496713,2.906125,2.428623,1.086481,0.740760,0.490087,0.496176,-0.203105,2.965993,2.313567,0.751024,-0.419099,1.234938,1.234740,2.691862,2.745512,2.983514
0021800084,0.865846,2.335159,-0.609772,0.601267,1.399546,-0.724321,-1.728439,-1.633623,-0.546880,1.726395,-1.482996,-0.275425,0.976331,-0.419099,0.548861,-0.759840,-0.299096,0.123394,-1.452842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0021800877,-1.295454,0.420282,-1.572074,0.000000,0.101784,-0.130853,-1.063575,-0.675502,-1.149042,-1.506315,-1.280770,-1.882384,0.276433,1.219280,-0.344101,0.844416,0.450898,-1.598131,-2.515335
0021800879,-0.948993,1.013852,-1.797975,-0.844605,0.675744,-3.199351,-2.065649,-2.026507,-0.164591,0.645564,-0.795834,-0.248494,-0.760601,-0.155108,-1.040949,-1.581764,-1.644112,-2.209827,-0.672734
0021800884,-2.302847,-1.200863,-1.518937,-2.073311,-0.472251,-2.458826,-0.593618,-0.533497,-0.444038,2.039085,0.079712,1.242470,-3.187951,1.398082,0.693966,-1.584615,-1.327503,-2.925044,-0.223393


In [229]:
#Fit model
clf = LogisticRegression(random_state=0).fit(train_X, train_Y)
y_pred = clf.predict(test_X)

In [230]:
#Train accuracy
clf.score(train_X, train_Y)

0.6427507447864945

In [231]:
#Test accuracy
accuracy_score(test_Y, y_pred)

0.6514186450492183

In [154]:
clf.predict([[-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3]])

c:\users\batar\desktop\nba\env\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


array(['L'], dtype=object)

In [232]:
train_X

,FGM_H,FGA_H,FG_PCT_H,FG3M_H,FG3A_H,FG3_PCT_H,FTM_H,FTA_H,FT_PCT_H,OREB_H,DREB_H,REB_H,AST_H,STL_H,BLK_H,TOV_H,PF_H,PTS_H,PLUS_MINUS_H
GAME_ID,,,,,,,,,,,,,,,,,,,
0021800068,-1.858243,1.640968,-3.462870,0.201080,0.573655,-0.711188,-0.804507,-0.219714,-1.641395,0.637193,-1.279685,-0.731177,-3.640845,0.281690,-2.056876,-2.693408,-1.546329,-2.066312,-1.812867
0021800077,-1.238829,0.717924,-2.150148,-2.111338,-0.614630,-3.437410,2.042210,1.483069,2.177868,1.699182,-1.010277,0.056244,-1.040241,-0.845070,-2.605376,-0.092876,0.483228,-0.762946,-2.825940
0021800081,1.598485,0.456879,1.496713,2.906125,2.428623,1.086481,0.740760,0.490087,0.496176,-0.203105,2.965993,2.313567,0.751024,-0.419099,1.234938,1.234740,2.691862,2.745512,2.983514
0021800078,-0.333018,0.203057,-0.554338,-0.701479,-0.123489,-0.946791,0.185190,0.816811,-1.256738,-1.117079,2.359312,1.322038,0.976331,0.139700,-0.960508,0.569880,1.296082,-0.431878,-1.375011
0021800083,-2.331124,-0.253822,-2.480664,0.400845,0.699773,-0.936443,-0.925949,-0.381179,-1.828970,1.320184,0.606680,1.211868,-1.727355,-1.536697,-0.137215,1.424700,-1.196383,-2.498724,-1.971713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0021800645,-1.466357,-1.912893,-0.690240,-0.534617,0.250414,-0.873501,1.764610,1.894824,-0.186583,0.250958,0.659658,0.671938,-1.955377,-1.974943,-0.314793,2.989277,0.932523,-0.653074,-1.184914
0021800651,1.975342,2.212828,0.543502,0.000000,0.177622,-0.640411,-0.739791,-0.594039,0.098615,1.178429,0.640446,1.050695,0.742920,-2.600133,-4.389910,-1.416180,-3.667240,1.273450,-0.112262
0021800654,1.633181,0.851794,1.425785,0.000000,-0.185448,0.300486,1.480845,0.442876,2.828255,-1.137670,0.724812,-0.058914,0.238131,1.245639,-0.920370,0.728411,1.081023,2.124384,0.798842


In [ ]:
#Next steps
#See if more data helps
#See if more attributes help (advanced stats)
#See if different rolling averages have different performance

In [112]:
stats_df.iloc[:, [20,43]]

,TEAM_NAME_H,TEAM_NAME_A
GAME_ID,,
0021800068,New York Knicks,Golden State Warriors
0021800078,Philadelphia 76ers,Charlotte Hornets
0021800077,Cleveland Cavaliers,Indiana Pacers
0021800081,Milwaukee Bucks,Orlando Magic
0021800084,Dallas Mavericks,Utah Jazz
...,...,...
0021801225,Memphis Grizzlies,Golden State Warriors
0021801228,Denver Nuggets,Minnesota Timberwolves
0021801224,Philadelphia 76ers,Chicago Bulls


In [18]:
stats_df.columns

Index(['FGM_H', 'FGA_H', 'FG_PCT_H', 'FG3M_H', 'FG3A_H', 'FG3_PCT_H', 'FTM_H',
       'FTA_H', 'FT_PCT_H', 'OREB_H', 'DREB_H', 'REB_H', 'AST_H', 'STL_H',
       'BLK_H', 'TOV_H', 'PF_H', 'PTS_H', 'PLUS_MINUS_H', 'GAME_DATE_H',
       'TEAM_NAME_H', 'MATCHUP_H', 'WL_H', 'FGM_A', 'FGA_A', 'FG_PCT_A',
       'FG3M_A', 'FG3A_A', 'FG3_PCT_A', 'FTM_A', 'FTA_A', 'FT_PCT_A', 'OREB_A',
       'DREB_A', 'REB_A', 'AST_A', 'STL_A', 'BLK_A', 'TOV_A', 'PF_A', 'PTS_A',
       'PLUS_MINUS_A', 'GAME_DATE_A', 'TEAM_NAME_A', 'MATCHUP_A', 'WL_A'],
      dtype='object')

In [11]:
game_means, game_stds = generate_population_statistics(stats_df)

In [12]:
game_means

array([[ 4.11000000e+01,  9.03400000e+01,  4.55646667e-01, ...,
         2.26533333e+01,  1.12140000e+02, -8.00000000e-02],
       [ 4.11000000e+01,  9.03400000e+01,  4.55646667e-01, ...,
         2.26533333e+01,  1.12140000e+02, -8.00000000e-02],
       [ 4.11000000e+01,  9.03400000e+01,  4.55646667e-01, ...,
         2.26533333e+01,  1.12140000e+02, -8.00000000e-02],
       ...,
       [ 4.23333333e+01,  9.05266667e+01,  4.68820000e-01, ...,
         1.94400000e+01,  1.13126667e+02,  2.53333333e-01],
       [ 4.23333333e+01,  9.05266667e+01,  4.68820000e-01, ...,
         1.94400000e+01,  1.13126667e+02,  2.53333333e-01],
       [ 4.23333333e+01,  9.05266667e+01,  4.68820000e-01, ...,
         1.94400000e+01,  1.13126667e+02,  2.53333333e-01]])

In [118]:
final, matchup = normalize_and_standardize(stats_df, game_means, game_stds)

C:\Users\batar\AppData\Local\Temp\ipykernel_8300\1595565284.py:17: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  final = np.subtract(home_z, away_z)


In [119]:
matchup

,TEAM_NAME_A,TEAM_NAME_H
GAME_ID,,
0021800068,Golden State Warriors,New York Knicks
0021800078,Charlotte Hornets,Philadelphia 76ers
0021800077,Indiana Pacers,Cleveland Cavaliers
0021800081,Orlando Magic,Milwaukee Bucks
0021800084,Utah Jazz,Dallas Mavericks
...,...,...
0021801225,Golden State Warriors,Memphis Grizzlies
0021801228,Minnesota Timberwolves,Denver Nuggets
0021801224,Chicago Bulls,Philadelphia 76ers


In [ ]:
#Prepare training and test data



In [ ]:
stats_arr

In [ ]:
#Function to flatten stats dict and find averages. Would show average and std dev. of team atttributes for 5th, 
# 6th... 82nd game

num_teams = teams.size

In [ ]:
#test
team = 'Oklahoma City Thunder'
temp_df = df[df['TEAM_NAME'] == team]
temp_df = temp_df.sort_values(by = ['GAME_DATE'])
temp_df = temp_df.set_index('GAME_ID')
temp_df = temp_df[[
           'FGM', 'FGA', 'FG_PCT', 'FG3M',
           'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
           'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']]
final_stats_df = temp_df.rolling(5).mean().shift(periods = 1).iloc[5:]
single_stats_arr = final_stats_df.to_numpy()
all_stats_arr = np.empty((30, 77, 19))


In [ ]:
stats_calc_dict = {}
descriptive_dict = {}
all_stats_arr = np.empty((30, 77, 19))
team = 'Oklahoma City Thunder'
temp_df = df[df['TEAM_NAME'] == team]
temp_df = temp_df.sort_values(by = ['GAME_DATE'])
temp_df = temp_df.set_index('GAME_ID')
key_data = temp_df[['TEAM_NAME', 'MATCHUP', 'WL']].iloc[5:]
temp_df = temp_df[[
           'FGM', 'FGA', 'FG_PCT', 'FG3M',
           'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
           'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']]
final_stats_df = temp_df.rolling(5).mean().shift(periods = 1).iloc[5:]
single_stats_arr = final_stats_df.to_numpy()
all_stats_arr[0] = single_stats_arr
        
final_organized_stats = temp_df.rolling(5).mean().shift(periods = 1).iloc[5:]
stats_with_key = pd.concat([final_organized_stats, key_data], axis = 1)

In [ ]:
all_teams_stats_df = pd.DataFrame()

In [ ]:
all_teams_stats_df = pd.concat([all_teams_stats_df, stats_with_key], axis = 0)

In [ ]:
temp_df = df[df['TEAM_NAME'] == team]
temp_df.head(10)

In [ ]:
stats_with_key

In [ ]:
final_organized_stats = temp_df.rolling(5).mean().shift(periods = 1).iloc[5:]

In [ ]:
final_organized_stats

In [ ]:
final_stats_df.to_numpy().shape

In [ ]:
df.from_dict(dict2['Oklahoma City Thunder'], orient = 'index')

In [ ]:
dict1, dict2 = generate_team_stats(df, teams)

In [ ]:
dict2

In [ ]:
temp_df = df[df['TEAM_NAME'] == 'Oklahoma City Thunder']

In [ ]:
temp_df = temp_df.set_index('GAME_ID')

In [ ]:
game_ids = temp_df['GAME_ID']

In [ ]:
temp_df = temp_df[[
           'FGM', 'FGA', 'FG_PCT', 'FG3M',
           'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
           'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']]
temp_df.rolling(5).mean().shift(periods = 1).iloc[5:]

In [ ]:
game_ids.shift(periods = 5).iloc[5:]

In [ ]:
temp_df = temp_df.sort_values(by = ['GAME_DATE'])

In [ ]:
temp_df = temp_df[[
       'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']]

In [ ]:
temp_df.rolling(5).mean().shift(periods = 1).tail(10)

In [ ]:
df.head(5)

In [ ]:
#Split data frames into home and away teams
home_df = df[df['MATCHUP'].str.contains("vs.")]
away_df = df[df['MATCHUP'].str.contains("@")]

In [ ]:
merged = home_df.merge(away_df, on = "GAME_ID",suffixes = ("_H", "_A"))

In [ ]:
merged

In [ ]:
#make list of all team names, for loop filter by team n times, and make the rolling dict values?
# Can then go in and replace values/make new df with the normalized vals